In [ ]:
!pip install protobuf>=5.29.1
!pip install git+https://github.com/huggingface/parler-tts.git
!pip install flask flask_cors pyngrok soundfile

  Cloning https://github.com/huggingface/parler-tts.git to /tmp/pip-req-build-05i1zgsj
  Running command git clone --filter=blob:none --quiet https://github.com/huggingface/parler-tts.git /tmp/pip-req-build-05i1zgsj
  Resolved https://github.com/huggingface/parler-tts.git to commit d108732cd57788ec86bc857d99a6cabd66663d68
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Cloning https://github.com/descriptinc/audiotools to /tmp/pip-install-sfdy3b9z/descript-audiotools_118b97083cf24e3ba22a554571b74a12
  Running command git clone --filter=blob:none --quiet https://github.com/descriptinc/audiotools /tmp/pip-install-sfdy3b9z/descript-audiotools_118b97083cf24e3ba22a554571b74a12
  Resolved https://github.com/descriptinc/audiotools to commit 348ebf2034ce24e2a91a553e3171cb00c0c71678
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.1/44.1 kB 3.8 MB/s eta 0:00:00
  Prep

In [ ]:
from flask import Flask, request, send_file, jsonify
from flask_cors import CORS
from parler_tts import ParlerTTSForConditionalGeneration
from transformers import AutoTokenizer
import torch
import soundfile as sf
import numpy as np
from pyngrok import ngrok
import os

In [ ]:
app = Flask(__name__)
CORS(app)

In [ ]:
device = "cuda" if torch.cuda.is_available() else "cpu"
print("Using device:", device)

Using device: cuda


In [ ]:
print("Loading model...")
model = ParlerTTSForConditionalGeneration.from_pretrained("ai4bharat/indic-parler-tts").to(device)
tokenizer = AutoTokenizer.from_pretrained("ai4bharat/indic-parler-tts")
description_tokenizer = AutoTokenizer.from_pretrained(model.config.text_encoder._name_or_path)
print("Model loaded.")

Loading model...


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/7.34k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/3.75G [00:00<?, ?B/s]

  "_name_or_path": "google/flan-t5-large",
  "architectures": [
    "T5ForConditionalGeneration"
  ],
  "classifier_dropout": 0.0,
  "d_ff": 2816,
  "d_kv": 64,
  "d_model": 1024,
  "decoder_start_token_id": 0,
  "dense_act_fn": "gelu_new",
  "dropout_rate": 0.1,
  "eos_token_id": 1,
  "feed_forward_proj": "gated-gelu",
  "initializer_factor": 1.0,
  "is_encoder_decoder": true,
  "is_gated_act": true,
  "layer_norm_epsilon": 1e-06,
  "model_type": "t5",
  "n_positions": 512,
  "num_decoder_layers": 24,
  "num_heads": 16,
  "num_layers": 24,
  "output_past": true,
  "pad_token_id": 0,
  "relative_attention_max_distance": 128,
  "relative_attention_num_buckets": 32,
  "tie_word_embeddings": false,
  "transformers_version": "4.46.1",
  "use_cache": true,
  "vocab_size": 32128
}

  "_name_or_path": "ylacombe/dac_44khz",
  "architectures": [
    "DacModel"
  ],
  "codebook_dim": 8,
  "codebook_loss_weight": 1.0,
  "codebook_size": 1024,
  "commitment_loss_weight": 0.25,
  "decoder_hidden_si

...

tokenizer_config.json:   0%|          | 0.00/2.54k [00:00<?, ?B/s]

spiece.model:   0%|          | 0.00/792k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/2.42M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/2.20k [00:00<?, ?B/s]

Model loaded.


In [ ]:
@app.route('/tts', methods=['POST'])
def tts():
    data = request.get_json()
    prompt = data.get('prompt', '')
    description = data.get('description', '')

    if not prompt or not description:
        return jsonify({"error": "Missing prompt or description"}), 400

    try:
        desc_ids = description_tokenizer(description, return_tensors="pt").to(device)
        prompt_ids = tokenizer(prompt, return_tensors="pt").to(device)

        with torch.no_grad():
            generation = model.generate(
                input_ids=desc_ids.input_ids,
                attention_mask=desc_ids.attention_mask,
                prompt_input_ids=prompt_ids.input_ids,
                prompt_attention_mask=prompt_ids.attention_mask
            )
        audio_arr = generation.cpu().numpy().squeeze()
        output_path = "indic_tts_output.wav"
        sf.write(output_path, audio_arr, model.config.sampling_rate)

        return send_file(output_path, mimetype="audio/wav")

    except Exception as e:
        return jsonify({"error": str(e)}), 500

In [ ]:
!ngrok config add-authtoken 2vtEFj0tGUjOXWdKpEzRw0lm9hI_6aATradqDJhfp3iB9SjK1

Authtoken saved to configuration file: /root/.config/ngrok/ngrok.yml


In [ ]:
port = 5005
public_url = ngrok.connect(port)
print("🔥 Public TTS API:", public_url)
app.run(port=port)

🔥 Public TTS API: NgrokTunnel: "https://bbeb-34-126-182-31.ngrok-free.app" -> "http://localhost:5005"
